<a href="https://colab.research.google.com/github/webbigdata-jp/python_sample/blob/main/C3TR_Adapter_gguf_Free_Colab_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

C3TR-Adapter_ggufは日英・英日に特化した機械翻訳モデル[webbigdata/C3TR-Adapter](https://huggingface.co/webbigdata/C3TR-Adapter)をgpuがなくても動くように変換したサンプルです。  このColabのサンプルに従うと、GPUがないMacやColabでも動かす事ができます。    

C3TR-Adapter_gguf is a sample that converts a machine translation model[webbigdata/C3TR-Adapter](https://huggingface.co/webbigdata/C3TR-Adapter) specialized for Japanese-English and English-Japanese to run without a gpu.  If you follow this Colab sample, you can run it on a Mac or Colab without a GPU.


llama.cppを使うと、様々な量子化手法でファイルのサイズを小さくする事が出来ます。本サンプルでは5種類のみを扱います。小さいサイズのモデルは、少ないメモリで高速に動作させることができますが、モデルの性能も低下します。4ビット(q4_0)くらいがバランスが良いと言われていますが、本サンプルコードでは特定の文章を全モデルで翻訳し、どのモデルが貴方の作業に適切かを確認できるようにしたものです。gguf版はパラメーターの調整をしないと幻覚が増えます。  

Although llama.cpp can be used to reduce the size of the file with various quantization methods, this sample deals with only 5 types. Smaller models can run faster with less memory, but the performance of the models is also reduced. 4 bits (q4_0) is said to be a good balance, but this sample code translates a particular sentence with all models so that you can see which model is appropriate for your work. This sample code translates a specific sentence across all models so that you can see which model is appropriate for your work.The GGUF version will have more hallucinations if you don't adjust the parameters.  

- C3TR-Adapter.Q4_0.gguf 5.01 GB
- C3TR-Adapter.Q4_1.gguf 5.5 GB
- C3TR-Adapter.Q5_0.gguf 5.98 GB
- C3TR-Adapter.Q5_1.gguf 6.47 GB
- C3TR-Adapter.IQ1_S.gguf 2.16 GB (1bit量子化。正常動作しません。1-bit quantization still does not work as intended)


必要に応じて以下のパラメーターを調整してください  

- 温度（--temp）: この値を下げると、モデルがより確信度の高い（つまり、より一般的な）単語を選択する傾向が強くなります。
- トップP（--top_p）: この値をさらに低く設定することで、モデルが考慮する単語の範囲を狭め、より一貫性のあるテキストを生成するようになります。
- 生成する単語数（-n）: この値を減らすことで、モデルが生成するテキストの長さを短くし、不要な追加テキストの生成を防ぐことができます。-1 = 無限大、-2 = 文脈が満たされるまで。

以下はllama.cppの作者(ggerganov)による推奨パラメーターです

- -e (改行\nをエスケープ)
- --temp 0 (最も確率の高いトークンのみを選択)
- --repeat-penalty 1.0 (繰り返しペナルティをオフ)

Adjust the following parameters as needed  
- Temperature (--temp): Lowering this value will make the model more likely to select more confident (i.e., more common) words.
- Top P (--top_p): Setting this value even lower will narrow the range of words considered by the model and produce more consistent text.
- Number of words to generate (-n): Reducing this value will shorten the length of text generated by the model and prevent the generation of unnecessary additional text. -1 = infinity(default), -2 = until context filled.

The following are the recommended parameters by the author of llama.cpp(ggerganov)
- -e (escape newlines (\n))
- --temp 0(pick most probable tokens)
- --repeat-penalty 1.0(disable repetition penalty (it's never a good idea to have this with instruction tuned models)


上段メニューより「ランタイム」→「全てのセルを実行」で実行できます。  
You can run this by selecting "Runtime" > "Run All Cells" from the top menu.  

In [ ]:
import os
import subprocess

os.environ['TRANSLATE_TEMPLATE_Ja'] = """### Instructions:
Translate Japanese to English.
### Input:
人の一生は重荷を負うて遠き道を行くがごとし。急ぐべからず。不自由を常と思えば不足なし。
### Answer:
"""

os.environ['TRANSLATE_TEMPLATE_En'] = """### Instructions:
Translate English to Japanese.
### Input:
The person providing for own improvement should have no time when he quarrels.
### Answer:
"""

os.environ['BASE_URL'] = "https://huggingface.co/webbigdata/C3TR-Adapter_gguf/resolve/main/C3TR-Adapter."

In [ ]:
%%bash
# setup llama.cpp
git clone https://github.com/ggerganov/llama.cpp.git
cd llama.cpp
make -j

I ccache not found. Consider installing it for faster compilation.
I llama.cpp build info: 
I UNAME_S:   Linux
I UNAME_P:   x86_64
I UNAME_M:   x86_64
I CFLAGS:    -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG  -std=c11   -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -pthread -march=native -mtune=native -Wdouble-promotion 
I CXXFLAGS:  -std=c++11 -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations -Wmissing-noreturn -pthread  -march=native -mtune=native -Wno-array-bounds -Wno-format-truncation -Wextra-semi -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG 
I NVCCFLAGS: -std=c++11 -O3 
I LDFLAGS:    
I CC:        cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
I CXX:       g++ (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0

cc  -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG  -std=c11   -fPIC -O3

Cloning into 'llama.cpp'...


## C3TR-Adapter.Q4_0.gguf

In [ ]:
%%bash
cd llama.cpp
wget ${BASE_URL}Q4_0.gguf 2>>q4_0.log

In [ ]:
%%bash
cd llama.cpp && ./main -e --temp 0 --repeat-penalty 1.0 -n -2 -m 'C3TR-Adapter.Q4_0.gguf' -p "$TRANSLATE_TEMPLATE_Ja" 2>q4_0.log

 ### Instructions:
Translate Japanese to English.
### Input:
人の一生は重荷を負うて遠き道を行くがごとし。急ぐべからず。不自由を常と思えば不足なし。
### Answer:
A man's life is like a journey in which he must carry a heavy burden. He must not hurry. If he always expects trouble, he will never be disappointed.


In [ ]:
%%bash
cd llama.cpp && ./main -e --temp 0 --repeat-penalty 1.0 -n -2 -m 'C3TR-Adapter.Q4_0.gguf' -p "$TRANSLATE_TEMPLATE_En" 2>>q4_0.log

 ### Instructions:
Translate English to Japanese.
### Input:
The person providing for own improvement should have no time when he quarrels.
### Answer:
自分の向上に努めている人は、けんかをする暇がないはずだ。


## C3TR-Adapter.Q4_1.gguf

In [ ]:
%%bash
cd llama.cpp
wget ${BASE_URL}Q4_1.gguf 2>>q4_1.log

In [ ]:
%%bash
cd llama.cpp && ./main -e --temp 0 --repeat-penalty 1.0 --no-penalize-nl -n -2 -m 'C3TR-Adapter.Q4_1.gguf' -p "$TRANSLATE_TEMPLATE_Ja" 2>q4_1.log

 ### Instructions:
Translate Japanese to English.
### Input:
人の一生は重荷を負うて遠き道を行くがごとし。急ぐべからず。不自由を常と思えば不足なし。
### Answer:
A man's life is like a journey in which he bears a heavy burden. He should not hurry. If he always thinks of his troubles, he will never lack them.


In [ ]:
%%bash
cd llama.cpp && ./main -e --temp 0 --repeat-penalty 1.0 --no-penalize-nl -n -2 -m 'C3TR-Adapter.Q4_1.gguf' -p "$TRANSLATE_TEMPLATE_En" 2>>q4_1.log

 ### Instructions:
Translate English to Japanese.
### Input:
The person providing for own improvement should have no time when he quarrels.
### Answer:
自分の向上に努める人は、けんかをする時間がないはずだ。


## C3TR-Adapter.Q5_0.gguf

In [ ]:
%%bash
cd llama.cpp
wget ${BASE_URL}Q5_0.gguf 2>>q5_0.log

In [ ]:
%%bash
cd llama.cpp && ./main -e --temp 0 --repeat-penalty 1.0 --no-penalize-nl -n -2 -m 'C3TR-Adapter.Q5_0.gguf' -p "$TRANSLATE_TEMPLATE_Ja" 2>q5_0.log

 ### Instructions:
Translate Japanese to English.
### Input:
人の一生は重荷を負うて遠き道を行くがごとし。急ぐべからず。不自由を常と思えば不足なし。
### Answer:
A man's life is like a journey with a heavy burden. No need to hurry. If you expect trouble, you'll never be disappointed.


In [ ]:
%%bash
cd llama.cpp && ./main -e --temp 0 --repeat-penalty 1.0 --no-penalize-nl -n -2 -m 'C3TR-Adapter.Q5_0.gguf' -p "$TRANSLATE_TEMPLATE_En" 2>>q5_0.log

 ### Instructions:
Translate English to Japanese.
### Input:
The person providing for own improvement should have no time when he quarrels.
### Answer:
自分の向上に努める人は、争う時間にしないべきだ。


## C3TR-Adapter.Q5_1.gguf

In [ ]:
!cd llama.cpp && wget ${BASE_URL}Q5_1.gguf 2>>q5_1.log

In [ ]:
!cd llama.cpp && ./main -e --temp 0 --repeat-penalty 1.0 --no-penalize-nl -n -2 -m 'C3TR-Adapter.Q5_1.gguf' -p "$TRANSLATE_TEMPLATE_Ja" 2>q5_1.log

 ### Instructions:
Translate Japanese to English.
### Input:
人の一生は重荷を負うて遠き道を行くがごとし。急ぐべからず。不自由を常と思えば不足なし。
### Answer:
A man's life is like a long journey with a heavy burden. He should not hurry. If he always expects trouble, he will never be disappointed.


In [ ]:
!cd llama.cpp && ./main -e --temp 0 --repeat-penalty 1.0 --no-penalize-nl -n -2 -m 'C3TR-Adapter.Q5_1.gguf' -p "$TRANSLATE_TEMPLATE_En" 2>>q5_1.log

 ### Instructions:
Translate English to Japanese.
### Input:
The person providing for own improvement should have no time when he quarrels.
### Answer:
自分の向上に努める人は、争う暇がないはずだ。


## C3TR-Adapter.IQ1_S.gguf

In [ ]:
!cd llama.cpp && wget ${BASE_URL}IQ1_S.gguf 2>iq1_s.log

In [ ]:
!cd llama.cpp && ./main -e --temp 0 --repeat-penalty 1.0 --no-penalize-nl -n 20 -m 'C3TR-Adapter.IQ1_S.gguf' -p "$TRANSLATE_TEMPLATE_Ja" 2>>iq1_s.log

 ### Instructions:
Translate Japanese to English.
### Input:
人の一生は重荷を負うて遠き道を行くがごとし。急ぐべからず。不自由を常と思えば不足なし。
### Answer:
????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????

In [ ]:
!cd llama.cpp && ./main -e --temp 0 --repeat-penalty 1.0 --no-penalize-nl -n 30  -m 'C3TR-Adapter.IQ1_S.gguf' -p "$TRANSLATE_TEMPLATE_En" 2>>iq1_s.log

 ### Instructions:
Translate English to Japanese.
### Input:
The person providing for own improvement should have no time when he quarrels.
### Answer:





 mlb mlb mlb mlb mlb mlb mlb mlb mlb mlb mlb mlb mlb mlb mlb mlb mlb mlb mlb mlb mlb mlb mlb mlb mlb

## 謝辞 Acknowledgment

Original Base Model  
[google/gemma-7b](https://huggingface.co/google/gemma-7b)  

Base Model  
[unsloth/gemma-7b-bnb-4bit](https://huggingface.co/unsloth/gemma-7b-bnb-4bit)  

Original QLoRA Adapter  
[webbigdata/C3TR-Adapter](https://huggingface.co/webbigdata/C3TR-Adapter)  

This gguf model  
[webbigdata/C3TR-Adapter_gguf](https://huggingface.co/webbigdata/C3TR-Adapter_gguf)  

gguf parameters sample  
[gemma-7b-it/discussions](https://huggingface.co/google/gemma-7b-it/discussions/38#65d7b14adb51f7c160769fa1)


This work was done by : [webbigdata](https://webbigdata.jp/)